# Data Analysis and Visualizations

## Import Packages and Read in Database as Pandas DataFrame
We take the cleaned database and read it as a dataframe by merging our three tables together on the link variable. We also expand on columns which originally contained lists by seperating the strings by our delimiter "%%%".

In [30]:
import os
import sqlite3
import pandas as pd

In [31]:
DATA_FOLDER = os.path.join('../data/clean/')
conn = sqlite3.connect(os.path.join(DATA_FOLDER, 'moviedatabase.db'))

In [32]:
top_df = (
    pd.read_sql('SELECT * FROM movies', conn)
        .merge(pd.read_sql('SELECT * FROM technicalities', conn), 
                left_on='link', 
                right_on='link', 
                how='left')
        .merge(pd.read_sql('SELECT * FROM post_release', conn), 
                left_on='link', 
                right_on='link', 
                how='left')
)

In [33]:
def crypted_to_list(list_value):
    return list_value.split("%%%")[1:]

In [34]:
columns_to_convert = ['Aspect Ratio', 'Director', 'Distributor', 'Genre', 
                      'Producer', 'Production Co', 'Rating', 'Screenwriter', 'Sound Mix']
for column in columns_to_convert:
    top_df[column] = top_df[column].apply(lambda x: crypted_to_list(x) if isinstance(x, str) else [])

In [35]:
top_df.head()

,link,title,Director,Genre,language,date-streaming,date-theater,Runtime,Rating,Aspect Ratio,Distributor,Sound Mix,Producer,Production Co,Screenwriter,rank,revenue,audience_score,critics_score
0,https://www.rottentomatoes.com/m/la_confidential,L.A. Confidential,[Curtis Hanson],"[Crime, Drama]",English,"Dec 12, 2015",1997-09-19,136,[R],"[35mm, Scope (2.35:1)]","[Warner Home Vídeo, Warner Bros.]","[Surround, DTS, Dolby Digital]","[Michael G. Nathanson, Arnon Milchan, Curtis H...","[Warner Brothers, Regency Enterprises]","[Curtis Hanson, James Ellroy, Brian Helgeland]",1,64600000.0,94,99
1,https://www.rottentomatoes.com/m/the_godfather,The Godfather,[Francis Ford Coppola],"[Crime, Drama]",English,"Aug 1, 2013",1972-03-15,177,[R],[Flat (1.85:1)],[Paramount Pictures],[Mono],[Albert S. Ruddy],[Paramount Pictures],"[Francis Ford Coppola, Mario Puzo, Mario Puzo]",2,134800000.0,98,97
2,https://www.rottentomatoes.com/m/1003707-casab...,Casablanca,[Michael Curtiz],[Drama],English,"Aug 15, 2008",1943-01-23,102,[PG],[Flat (1.37:1)],[Warner Bros. Pictures],[Mono],[Hal B. Wallis],[Warner Brothers],"[Murray Burnett, Joan Alison, Julius J. Epstei...",3,NaN,95,99
3,https://www.rottentomatoes.com/m/seven_samurai...,Seven Samurai,[Akira Kurosawa],[Action],Japanese,"Nov 29, 2011",1956-11-19,208,[],[Flat (1.37:1)],[Columbia Pictures],[Mono],[Sojiro Motoki],[Toho Company],"[Shinobu Hashimoto, Akira Kurosawa, Hideo Oguni]",4,192900.0,97,100
4,https://www.rottentomatoes.com/m/parasite_2019,Parasite,[Bong Joon Ho],"[Comedy, Mystery & Thriller, Drama]",Korean,"Oct 11, 2019",2019-11-01,132,"[R , Sexual Content, Language, Some Violence]",[Scope (2.35:1)],[Neon],"[Dolby Atmos, Dolby Digital]","[Kwak Sin-ae, Moon Yanggwon]",[Barunson E&A],"[Bong Joon Ho, Han Jinwon]",5,53400000.0,90,99


## Early Data Analysis
Play around and see any interesting trends or pieces of information

In [36]:
# Top directors here make sense, as each are notable figures within the film industry
top_df.explode('Director')['Director'].value_counts()

Alfred Hitchcock    7
Akira Kurosawa      6
Billy Wilder        6
Steven Spielberg    5
Martin Scorsese     4
                   ..
Guy Hamilton        1
George Lucas        1
John Crowley        1
Jon Watts           1
Robert Wiene        1
Name: Director, Length: 228, dtype: int64

In [37]:
# Andrew Stanton and Pete Docter both worked on Pixar films; Akira Kurosawa directed and wrote his screenplays
top_df.explode('Screenwriter')['Screenwriter'].value_counts()

Andrew Stanton        8
Pete Docter           6
Akira Kurosawa        6
Billy Wilder          6
Charlie Chaplin       4
                     ..
Guillermo del Toro    1
Robert Shaw           1
Howard Sackler        1
Carl Gottlieb         1
Hans Janowitz         1
Name: Screenwriter, Length: 528, dtype: int64

In [38]:
# We see that there are films with multiple genres, with drama, comedy, adventure being the most popular
top_df.explode('Genre')['Genre'].value_counts()

Drama                 159
Comedy                 88
Adventure              61
Mystery & Thriller     51
Fantasy                41
Action                 38
Crime                  34
Kids & Family          29
Romance                27
Animation              26
History                22
Sci-Fi                 22
Horror                 15
War                    13
Biography              13
Musical                10
Western                 8
Holiday                 5
LGBTQ+                  5
Music                   4
Anime                   3
Sports                  1
Documentary             1
Name: Genre, dtype: int64

In [39]:
# make revenues in millions to read numbers easier
top_df['revenue'] = top_df['revenue'].apply(lambda x: x/1000000)
top_df.explode('Genre').groupby('Genre')['revenue'].describe().sort_values(by=['mean'])

,count,mean,std,min,25%,50%,75%,max
Genre,,,,,,,,
LGBTQ+,4.0,13.413800,14.490950,0.055200,2.86380,10.9000,21.450,31.8
Romance,17.0,14.384447,17.983562,0.017800,0.16260,4.7000,34.100,44.7
War,7.0,18.959257,29.871354,0.017800,0.04025,0.5165,26.650,78.8
Drama,100.0,47.280934,63.086114,0.006400,0.84970,20.4000,62.425,329.7
Sports,1.0,52.400000,NaN,52.400000,52.40000,52.4000,52.400,52.4
Music,4.0,56.772200,102.635983,0.188800,2.44720,8.2000,62.525,210.5
Crime,17.0,59.304159,56.860516,0.029600,7.60000,42.6000,124.100,164.4
Biography,13.0,69.061054,90.493913,0.006400,7.90000,38.9000,96.900,329.7
History,19.0,72.258405,88.068571,0.006400,0.25495,52.1000,126.800,329.7


In [40]:
top_df['critics_score'].describe()

count    300.000000
mean      96.546667
std        2.306271
min       89.000000
25%       95.000000
50%       97.000000
75%       98.000000
max      100.000000
Name: critics_score, dtype: float64

In [41]:
top_df['audience_score'].describe()

count    300.000000
mean      91.066667
std        3.711442
min       79.000000
25%       89.000000
50%       92.000000
75%       94.000000
max       99.000000
Name: audience_score, dtype: float64

Interestingly, we see that the audience scores have a higher standard deviation than the critics showing that critics have a more homogenous opinion than the audience. The fact that the mean is lower for the audience means that appreciate these movies less than the critics. 

In [42]:
genre_critics = top_df.explode('Genre').groupby('Genre')['critics_score'].describe().sort_values(by=['mean'])
genre_critics = genre_critics.drop(genre_critics[genre_critics['count'] < 10].index)
genre_critics.sort_values(by=['mean'])

,count,mean,std,min,25%,50%,75%,max
Genre,,,,,,,,
Sci-Fi,22.0,94.818182,2.461830,91.0,93.0,94.0,96.00,100.0
Biography,13.0,95.230769,2.554533,92.0,93.0,95.0,97.00,100.0
Fantasy,41.0,95.341463,2.186433,91.0,94.0,95.0,97.00,100.0
Action,38.0,95.342105,2.408053,91.0,94.0,95.0,97.00,100.0
Adventure,61.0,95.770492,2.333763,91.0,94.0,96.0,97.00,100.0
History,22.0,95.954545,2.256784,92.0,94.0,96.0,97.75,100.0
Horror,15.0,96.200000,1.780851,92.0,96.0,96.0,97.50,98.0
Crime,34.0,96.352941,2.484892,91.0,95.0,96.0,98.00,100.0
Animation,26.0,96.461538,1.964297,92.0,95.0,96.0,98.00,100.0


In [43]:
genre_audience = top_df.explode('Genre').groupby('Genre')['audience_score'].describe().sort_values(by=['mean'])
genre_audience = genre_audience.drop(genre_audience[genre_audience['count'] < 10].index)
genre_audience.sort_values(by=['mean'])

,count,mean,std,min,25%,50%,75%,max
Genre,,,,,,,,
Horror,15.0,89.466667,3.681356,83.0,87.00,89.0,93.00,95.0
Sci-Fi,22.0,89.727273,4.842095,79.0,86.25,91.0,93.50,97.0
Musical,10.0,90.100000,2.558211,87.0,89.00,89.5,91.50,95.0
Mystery & Thriller,51.0,90.254902,3.719372,79.0,88.00,90.0,93.00,96.0
Kids & Family,29.0,90.413793,2.639227,86.0,88.00,90.0,92.00,94.0
Romance,27.0,90.629630,3.236191,82.0,88.00,91.0,93.00,96.0
Animation,26.0,90.730769,2.807956,86.0,89.00,90.5,92.75,96.0
Action,38.0,90.763158,4.296033,79.0,88.00,90.5,94.00,99.0
Biography,13.0,90.923077,3.148056,86.0,89.00,91.0,93.00,98.0


Among genres, critics and the audience have noticeably different opinions. This is a point of interest I will explore later.

In [44]:
# Top grossing movies are all part of large franchises
top_df.sort_values(by=['revenue'], ascending=False).head()

,link,title,Director,Genre,language,date-streaming,date-theater,Runtime,Rating,Aspect Ratio,Distributor,Sound Mix,Producer,Production Co,Screenwriter,rank,revenue,audience_score,critics_score
141,https://www.rottentomatoes.com/m/avengers_endgame,Avengers: Endgame,"[Anthony Russo, Joe Russo]","[Action, Adventure, Fantasy, Sci-Fi]",English,"Jul 30, 2019",2019-04-26,181,"[PG-13 , Sequences of Sci-Fi Violence, Action,...",[Scope (2.35:1)],[Walt Disney],"[Dolby Atmos, DTS, Dolby Digital, SDDS]",[Kevin Feige],"[Marvel Studios, Walt Disney Pictures]","[Christopher Markus, Stephen McFeely]",142,858.4,90,94
127,https://www.rottentomatoes.com/m/spider_man_no...,Spider-Man: No Way Home,[Jon Watts],"[Action, Adventure, Fantasy, Comedy]",English,"Mar 15, 2022",2021-12-17,148,"[PG-13 , Sequences of Action/Violence, Brief S...",[Scope (2.35:1)],[Columbia Pictures],"[Dolby Digital, Dolby Atmos]","[Kevin Feige, Amy Pascal]","[Columbia Pictures, Pascal Pictures, Marvel St...","[Chris McKenna, Erik Sommers]",128,814.1,98,93
6,https://www.rottentomatoes.com/m/top_gun_maverick,Top Gun: Maverick,[Joseph Kosinski],"[Action, Adventure]",English,"Aug 22, 2022",2022-05-27,131,"[PG-13 , Some Strong Language, Sequences of In...",[Scope (2.35:1)],[Paramount Pictures],[Dolby Atmos],"[Jerry Bruckheimer, Tom Cruise, David Ellison,...","[Don Simpson/Jerry Bruckheimer Films, Paramoun...","[Ehren Kruger, Eric Warren Singer, Christopher...",7,718.5,99,96
224,https://www.rottentomatoes.com/m/black_panther...,Black Panther,[Ryan Coogler],"[Action, Adventure, Fantasy]",English,"May 2, 2018",2018-02-16,134,"[PG-13 , Sequences of Action Violence, A Brief...",[Scope (2.35:1)],[Walt Disney],"[DTS, Dolby Atmos]",[Kevin Feige],[Walt Disney Pictures],"[Ryan Coogler, Joe Robert Cole]",225,700.2,79,96
42,https://www.rottentomatoes.com/m/toy_story_4,Toy Story 4,[Josh Cooley],"[Kids & Family, Comedy, Adventure, Fantasy, An...",English,"Oct 1, 2019",2019-06-21,100,[G],[Scope (2.35:1)],[Walt Disney],"[DTS, Dolby Digital, Dolby Atmos]","[Mark Nielsen, Jonas Rivera, Galyn Susman]","[Pixar Animation Studios, Walt Disney Pictures]","[Andrew Stanton, Stephany Folsom]",43,434.0,94,97


In [45]:
# The two Star Wars movies seem to have incorrect revenue values as they are $123 and $345. Besides that, it's interesting to see "All About Eve", whiched ranked no 16 on the list, appear in the bottom 5 here
top_df.sort_values(by=['revenue']).head()

,link,title,Director,Genre,language,date-streaming,date-theater,Runtime,Rating,Aspect Ratio,Distributor,Sound Mix,Producer,Production Co,Screenwriter,rank,revenue,audience_score,critics_score
277,https://www.rottentomatoes.com/m/star_wars_epi...,Star Wars: The Force Awakens,[J.J. Abrams],"[Sci-Fi, Adventure, Action, Fantasy]",English,"Apr 1, 2016",2015-12-18,136,"[PG-13 , Sci-Fi Action Violence]",[Scope (2.35:1)],[Walt Disney],"[Dolby Digital, Dolby Atmos]","[Kathleen Kennedy, J.J. Abrams, Bryan Burk]","[Lucasfilm Ltd., Bad Robot]","[Lawrence Kasdan, J.J. Abrams, Michael Arndt]",278,0.000123,84,93
122,https://www.rottentomatoes.com/m/star_wars_epi...,Star Wars: Episode IV - A New Hope,[George Lucas],"[Sci-Fi, Adventure, Action, Fantasy]",English,"Oct 10, 2016",1977-05-25,121,[PG],[],[],[Surround],[Gary Kurtz],[Lucasfilm Ltd.],[George Lucas],123,0.000345,96,93
139,https://www.rottentomatoes.com/m/passion_of_jo...,The Passion of Joan of Arc,[Carl Theodor Dreyer],"[History, Drama, Biography]",None,"Jul 23, 2018",1928-04-21,77,[],[35mm],"[Criterion Collection, Video Yesteryear, Janus...",[],[],[Société générale des films],"[Carl Theodor Dreyer, Joseph Delteil]",140,0.006400,93,98
15,https://www.rottentomatoes.com/m/1000626-all_a...,All About Eve,[Joseph L. Mankiewicz],[Drama],English,"Mar 1, 2013",1950-10-13,138,[],[Flat (1.37:1)],"[20th Century Fox, Fox]",[Mono],[Darryl F Zanuck],[Twentieth Century Fox],[Joseph L. Mankiewicz],16,0.010200,94,99
61,https://www.rottentomatoes.com/m/pianist,The Pianist,[Roman Polanski],"[History, Drama, War]",English,"Apr 19, 2016",2002-12-27,148,"[R , Violence, Brief Strong Language]",[],[Focus Features],[Surround],"[Robert Benmussa, Roman Polanski, Alain Sarde]",[Miramax],"[Wladyslaw Szpilman, Ronald Harwood]",62,0.017800,96,95


In [46]:
# Run time of movies vary significantly and the longest movie is six times as long as the shortest.
top_df['Runtime'].describe()

count    300.000000
mean     117.700000
std       27.236914
min       60.000000
25%       97.750000
50%      113.000000
75%      131.000000
max      250.000000
Name: Runtime, dtype: float64

After collecting these initial impressions about the top movies lists, I want to explore the interplay between critic and audience scores, see what types of movies succeed, and how language and genre can have an affect on the reception of movies.

## Data Visualizations

In [47]:
# For plotting purposes
import math 
import cairosvg
from lets_plot import *
LetsPlot.setup_html()

## Analyzing the Interplay Between Critics Score and Audience Score

In [48]:
# Create new df to show genre behavior among films
# Also put log scale on revenue
# Interestingly, both Star Wars movie below have registered revenues in the US as $123 and $345
# As misinputed values, I remove
genre_df = top_df.explode('Genre')
genre_df['revenue'] = genre_df['revenue'].apply(lambda x: math.log10(x))
genre_df = genre_df.drop(genre_df[genre_df.title == "Star Wars: The Force Awakens"].index)
genre_df = genre_df.drop(genre_df[genre_df.title == "Star Wars: Episode IV - A New Hope"].index)


In [64]:
# Revenue vs audience score scatterplot
tooltip_setup1 = (
    layer_tooltips()
        .line('@title')
)

scatter_plot1 = (
    ggplot(genre_df, aes(x='revenue', y='audience_score', color='Genre')) +
    geom_jitter(tooltips=tooltip_setup1) +
    scale_x_continuous(name='Revenue') +
    scale_y_continuous(name='Audience Score', limits=[75, 100]) +
    labs(title='There Seems to be a Slight Correlation Between Revenue and Audience Score',
         x='Log 10 Revenue in Millions',
         y='Audience Score') +
    theme(
        axis_text_x=element_text(size=12),
        axis_text_y=element_text(size=12),
        axis_title_x=element_text(size=14),
        axis_title_y=element_text(size=14),
        plot_title=element_text(size=16, face='bold')
        
    ) +
    ggsize(750, 600)
)

scatter_plot1
ggsave(scatter_plot1, 'revenue_audience.png', path='../docs/figures')

'/Users/danielcho/Desktop/ME204/me204-2024-project-chodani/docs/figures/revenue_audience.png'

In [66]:
# Revenue vs critics score scatter plot
scatter_plot2 = (
    ggplot(genre_df, aes(x='revenue', y='critics_score', color='Genre')) +
    geom_jitter(tooltips=tooltip_setup1) +
    scale_x_continuous(name='Revenue') +
    scale_y_continuous(name='Critics Score', limits=[90, 100]) +
    labs(title='There is No Correlation Between Revenue and Critics Score',
         x='Log 10 Revenue in Millions',
         y='Critics Score') +
    theme(
        axis_text_x=element_text(size=12),
        axis_text_y=element_text(size=12),
        axis_title_x=element_text(size=14),
        axis_title_y=element_text(size=14),
        plot_title=element_text(size=16, face='bold')
        
    ) +
    ggsize(750,600)
)

scatter_plot2
ggsave(scatter_plot2, 'revenue_critics.png', path='../docs/figures')

'/Users/danielcho/Desktop/ME204/me204-2024-project-chodani/docs/figures/revenue_critics.png'

In [58]:
# Audience vs Critics Score
scatter_plot3 = (
    ggplot(top_df.explode('Genre'), aes(x='audience_score', y='critics_score', color='Genre')) +
    geom_jitter(tooltips=tooltip_setup1) +
    scale_x_continuous(name='Audience Score', limits=[75, 100]) +
    scale_y_continuous(name='Critics Score', limits=[90, 100]) +
    labs(title='There Seems to be no Linear Relationship',
         x='Audience Score',
         y='Critics Score') +
    theme(
        axis_text_x=element_text(size=12),
        axis_text_y=element_text(size=12),
        axis_title_x=element_text(size=14),
        axis_title_y=element_text(size=14),
        plot_title=element_text(size=16, face='bold')
        
    )
    + ggsize(750,600)
)

scatter_plot3
ggsave(scatter_plot3, 'audience_critics.png', path='../docs/figures')

'/Users/danielcho/Desktop/ME204/me204-2024-project-chodani/docs/figures/audience_critics.png'

In [67]:
# Find difference in mean rating between critics and audience within different genres
diff_df = genre_critics['mean'] - genre_audience['mean']
diff_df = diff_df.to_frame().reset_index().sort_values(by=['mean'])

scatter_plot4 = (
    ggplot(diff_df, aes(x='Genre', y='mean',)) +
    geom_point() +
    scale_x_continuous(name='Genre') +
    scale_y_continuous(name='Rating Difference') +
    labs(title='Musicals are Underpreciated among the Audience and War Films are almost Equally liked',
         x='Genre',
         y='Rating Differnce (Critics Mean - Audience Mean Score)') +
    theme(
        axis_text_x=element_text(size=12),
        axis_text_y=element_text(size=12),
        axis_title_x=element_text(size=14),
        axis_title_y=element_text(size=14),
        plot_title=element_text(size=16, face='bold')
        
    ) +
    ggsize(800,600)
)

scatter_plot4
ggsave(scatter_plot4, 'diff_mean_genre.png', path='../docs/figures')

'/Users/danielcho/Desktop/ME204/me204-2024-project-chodani/docs/figures/diff_mean_genre.png'

## A Brief Look at Language, Release Data, and Genre

In [70]:
# Plot rank vs revenue with categories on language
tooltip_setup2 = (
    layer_tooltips()
        .line('@title')
        .line('@language')
        .line('$@revenue m')
)

scatter_plot5 = (
    ggplot(top_df, aes(x='rank', y='revenue', color='language')) +
    geom_jitter(tooltips=tooltip_setup2) +
    scale_x_continuous(name='Rank') +
    scale_y_continuous(name='Revenue') +
    labs(title='Better Rank Does Not Indicate More Revenue',
         x='Rank',
         y='Revenue') +
    theme(
        axis_text_x=element_text(size=12),
        axis_text_y=element_text(size=12),
        axis_title_x=element_text(size=14),
        axis_title_y=element_text(size=14),
        plot_title=element_text(size=16, face='bold')
        
    ) +
    ggsize(750,500)
)

scatter_plot5
ggsave(scatter_plot5, 'rank_revenue.png', path='../docs/figures')

'/Users/danielcho/Desktop/ME204/me204-2024-project-chodani/docs/figures/rank_revenue.png'

In [68]:
# Rank vs runtime by language, excluding English as it overpopulates the list
language_df = top_df.drop(top_df[top_df.language == "English"].index)

tooltip_setup3 = (
    layer_tooltips()
        .line('@title')
        .line('@language')
)

scatter_plot6 = (
    ggplot(language_df, aes(x='rank', y='Runtime', color='language')) +
    geom_jitter(tooltips=tooltip_setup3) +
    scale_x_continuous(name='Rank') +
    scale_y_continuous(name='Run Time') +
    labs(title='Run Time Does Not Indicate Rank Nor is There a Clear Pattern with Languages',
         x='Rank',
         y='Run Time') +
    theme(
        axis_text_x=element_text(size=12),
        axis_text_y=element_text(size=12),
        axis_title_x=element_text(size=14),
        axis_title_y=element_text(size=14),
        plot_title=element_text(size=16, face='bold')
        
    ) +
    ggsize(750,500)
)

scatter_plot6
ggsave(scatter_plot6, 'rank_runtime.png', path='../docs/figures')

'/Users/danielcho/Desktop/ME204/me204-2024-project-chodani/docs/figures/rank_runtime.png'

In [62]:
# Plot date vs rank
date_df = top_df.sort_values(by=['date-theater'])

scatter_plot7 = (
    ggplot(date_df, aes(x='date-theater', y='rank', color='language')) +
    geom_point(tooltips=tooltip_setup1) +
    scale_x_datetime() +
    scale_y_continuous(name='Rank') +
    labs(title='Release Date has no Influence on Rank',
         x='Theater Release Date',
         y='Rank') +
    theme(
        axis_text_x=element_text(size=12),
        axis_text_y=element_text(size=12),
        axis_title_x=element_text(size=14),
        axis_title_y=element_text(size=14),
        plot_title=element_text(size=16, face='bold')
        
    ) +
    ggsize(750,500)
)

scatter_plot7
ggsave(scatter_plot7, 'date_rank.png', path='../docs/figures')

'/Users/danielcho/Desktop/ME204/me204-2024-project-chodani/docs/figures/date_rank.png'

In [63]:
# Plot distribution of runtime of movies based on genres

# Drop genres with less than 10 movies
genre_df = genre_df.drop(genre_df[genre_df.Genre == "Music"].index)
genre_df = genre_df.drop(genre_df[genre_df.Genre == "Anime"].index)
genre_df = genre_df.drop(genre_df[genre_df.Genre == "Sports"].index)
genre_df = genre_df.drop(genre_df[genre_df.Genre == "Documentary"].index)
genre_df = genre_df.drop(genre_df[genre_df.Genre == "LGBTQ+"].index)
genre_df = genre_df.drop(genre_df[genre_df.Genre == "Western"].index)
genre_df = genre_df.drop(genre_df[genre_df.Genre == "Holiday"].index)


p1 = (
    ggplot(genre_df, aes(x="Runtime", color="Genre")) +
    ggsize(1000, 500) +
    geom_density() + 
    labs(title='Distribution of Runtimes by Genre',
         subtitle="War Films are the Longest with Comedy being the Shortest") 
         )
p1
ggsave(p1, 'runtime_distribution_genre.png', path='../docs/figures')

'/Users/danielcho/Desktop/ME204/me204-2024-project-chodani/docs/figures/runtime_distribution_genre.png'